In [13]:
import os
import pandas as pd

# Path to the main directory
main_dir = "code-davinci-002/"

# List to store dataframes
dataframes = []

# Iterate over each subdirectory and process files
for sub_dir, _, files in os.walk(main_dir):
    if sub_dir == main_dir:
        continue  # Skip the main directory
    
    # List to store dataframes for the current subdirectory
    sub_dataframes = []
    
    for file in files:
        if file.endswith('.csv'):
            # Read CSV file
            file_path = os.path.join(sub_dir, file)
            df = pd.read_csv(file_path)
            
            # Append the dataframe to the list for the current subdirectory
            sub_dataframes.append(df)
    
    if sub_dataframes:
        # Concatenate dataframes within the current subdirectory
        sub_df = pd.concat(sub_dataframes, ignore_index=True)
        
        # Add a new column 'Name' with the folder name
        folder_name = os.path.basename(sub_dir)
        sub_df['Name'] = folder_name
        
        # Append the subdirectory dataframe to the main list
        dataframes.append(sub_df)

# Print or inspect the list of dataframes
# for df in dataframes:
#     print(f"Dataframe for folder: {df['Name'].iloc[0]}")
#     print(df.head())
#     print()

In [2]:
# Concatenate all dataframes into a single dataframe
final_df_1 = pd.concat(dataframes, ignore_index=True)

# Add a new column 'model' with the value 'code-davinci-002'
final_df_1['model'] = 'code-davinci-002'

In [3]:
final_df_1['Final Answer_0'].value_counts()[:30]

Final Answer_0
no       2348
yes      2173
(A)      1389
(B)      1381
(C)       922
error     829
(D)       428
True      425
2.0       422
4.0       341
5.0       332
7.0       323
8.0       314
3.0       302
1.0       301
False     300
6.0       284
10.0      281
12.0      270
9.0       249
14.0      217
20.0      198
15.0      192
30.0      172
16.0      168
11.0      166
18.0      164
(E)       158
13.0      144
17.0      139
Name: count, dtype: int64

In [4]:
final_df_1[final_df_1['Final Answer_0'] == '2']

,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,CoT_2,...,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39,Name,model


In [5]:
final_df_1['Final Answer_0'].value_counts()[-20:] # 

Final Answer_0
2.7199999999999998             1
653.0                          1
1.219879518072289              1
-265.0                         1
9.3                            1
2288.0                         1
398.0                          1
5808495.0                      1
-94.0                          1
433.0                          1
-65.0                          1
[53, 14]                       1
11820.0                        1
-194.0                         1
4.386363636363637              1
466.0                          1
804.0                          1
777622.0                       1
14415.0                        1
(A), (B), (C), (D), and (E)    1
Name: count, dtype: int64

In [6]:
final_df_1['Final Answer_0'].value_counts().index[8]

'2.0'

In [7]:
final_df_1['Final Answer_0'].value_counts().index[9]

'4.0'

In [8]:
def clean_answer_columns(df):
    # List of column names to clean
    answer_columns = [f"Final Answer_{i}" for i in range(40)] + ["Correct Answer"]
    
    # Iterate over each answer column
    for column in answer_columns:
        # Convert the column to string type
        df[column] = df[column].astype(str)
        
        # Iterate over each value in the column
        for i, value in enumerate(df[column]):
            try:
                # Check if the value can be converted to float
                float_value = float(value)
                
                # Format the float value to have one decimal place
                df.at[i, column] = f"{float_value:.1f}"
            except ValueError:
                # If the value cannot be converted to float, leave it as is
                pass
    
    return df

final_df_1 = clean_answer_columns(final_df_1)
final_df_1['Final Answer_0'].value_counts()[:20]

Final Answer_0
no       2348
yes      2173
(A)      1389
(B)      1381
(C)       922
error     829
(D)       428
2.0       426
True      425
4.0       343
5.0       336
7.0       323
8.0       314
3.0       306
1.0       301
False     300
6.0       285
10.0      284
12.0      273
9.0       249
Name: count, dtype: int64

In [15]:
final_df_1['Model'] = 'code-davinci-002'

In [18]:
final_df_1['Model']

0        code-davinci-002
1        code-davinci-002
2        code-davinci-002
3        code-davinci-002
4        code-davinci-002
               ...       
24060    code-davinci-002
24061    code-davinci-002
24062    code-davinci-002
24063    code-davinci-002
24064    code-davinci-002
Name: Model, Length: 24065, dtype: object

In [16]:
final_df_1.to_csv(main_dir+'cleaned.csv',index = False)

In [9]:
main_dir = "vicuna-13b/"

# List to store dataframes
dataframes = []

# Iterate over each subdirectory and process files
for sub_dir, _, files in os.walk(main_dir):
    if sub_dir == main_dir:
        continue  # Skip the main directory
    
    # List to store dataframes for the current subdirectory
    sub_dataframes = []
    
    for file in files:
        if file.endswith('.csv'):
            # Read CSV file
            file_path = os.path.join(sub_dir, file)
            try:
                df = pd.read_csv(file_path)
            except:
                print(file)
                break
            
            # Append the dataframe to the list for the current subdirectory
            sub_dataframes.append(df)
    
    if sub_dataframes:
        # Concatenate dataframes within the current subdirectory
        sub_df = pd.concat(sub_dataframes, ignore_index=True)
        
        # Add a new column 'Name' with the folder name
        folder_name = os.path.basename(sub_dir)
        sub_df['Name'] = folder_name
        
        # Append the subdirectory dataframe to the main list
        dataframes.append(sub_df)

salient_translation_seed2_cleaned.csv


In [10]:
len(dataframes)

13

In [11]:
# Concatenate all dataframes into a single dataframe
final_df_2 = pd.concat(dataframes, ignore_index=True)

# Add a new column 'model' with the value 'code-davinci-002'
final_df_2['model'] = 'vicuna-13b'
final_df_2= clean_answer_columns(final_df_2)

In [12]:
final_df_2['Final Answer_0'].value_counts()[:20]

Final Answer_0
no       4106
nan      3584
yes      1748
(A)       983
(B)       821
(C)       725
True      343
2.0       321
4.0       304
7.0       285
5.0       280
8.0       274
0.0       261
6.0       254
(D)       254
3.0       249
12.0      237
1.0       236
False     235
9.0       232
Name: count, dtype: int64

In [13]:
final_df = pd.concat([final_df_1,final_df_2]).reset_index(drop=True)

In [14]:
final_df_test = final_df.sample(1000)

In [15]:
import re
import numpy as np
def extract_len(df):
    step_count_buffer = []
    for col in df:
        if col.startswith('CoT_'):
            cleaned_answers = []
            for entry in df[col]:
                entry_str = str(entry)
                # Count the number of new lines in the entry
                num_of_newlines = entry_str.count('\n')
                
                if num_of_newlines == 0:
                    # If no new lines, count the number of sentences
                    num_of_sentences = entry_str.count('.') + 1
                    cleaned_answers.append(num_of_sentences)
                else:
                    cleaned_answers.append(num_of_newlines)
            
            step_count_buffer.append(cleaned_answers)
    
    step_count = np.array(step_count_buffer).T
    return step_count

extract_len(final_df_test)

array([[ 3,  3,  2, ...,  3,  2,  3],
       [ 3,  3,  2, ...,  4,  4,  3],
       [ 3,  2,  2, ...,  3,  3,  3],
       ...,
       [14, 14, 14, ..., 14, 14, 14],
       [ 3,  4,  3, ...,  3,  3,  3],
       [ 3,  2,  3, ...,  3,  3,  3]])

In [16]:
(extract_len(final_df_test)[:,0]==0).sum()

0